In [63]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow import data
from tensorflow.keras.layers import Resizing, RandomFlip, RandomRotation, Rescaling, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import image_dataset_from_directory
from PIL import Image
from pathlib import Path
from tensorflow.data import AUTOTUNE

In [55]:
# Params
folder_path = '../data'
batch_size = 32

In [54]:
def check_image_sizes(folder_path):
    # Get all subfolders and their contents
    folder = Path(folder_path)
    subfolders = [subfolder for subfolder in folder.iterdir() if subfolder.is_dir()]
    counter = 0
    # Get the size of the first image found
    first_image_size = None
    for subfolder in subfolders:
        images = list(subfolder.glob('*.jpg'))
        if images:
            with Image.open(images[0]) as img:
                first_image_size = img.size

            for image_path in images:
                with Image.open(image_path) as img:
                    if img.size != first_image_size:
                        # print(f"Image {image_path.name} in folder {subfolder.name} does not have the same size.")
                        counter += 1

    if counter != 0:
        print(f'{counter} images have a different size')
        return False
    print("All images have the same size.")
    return True

check_image_sizes(folder_path = folder_path)

956 images have a different size


/home/yjalil/.pyenv/versions/3.11.5/envs/belote/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:870: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/home/yjalil/.pyenv/versions/3.11.5/envs/belote/lib/python3.11/site-packages/PIL/JpegImagePlugin.py:853: UserWarning: Image appears to be a malformed MPO file, it will be interpreted as a base JPEG file
  warnings.warn(


False

In [56]:
train_ds = image_dataset_from_directory(
  Path(folder_path),
  validation_split=0.2,
  subset="training",
  seed=123,
  batch_size=batch_size)

Found 3199 files belonging to 32 classes.
Using 2560 files for training.


2024-02-28 13:23:31.201118: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-28 13:23:31.449054: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [57]:
val_ds = image_dataset_from_directory(
  Path(folder_path),
  validation_split=0.2,
  subset="validation",
  seed=123,
  batch_size=batch_size)

Found 3199 files belonging to 32 classes.
Using 639 files for validation.


In [62]:
class_names = train_ds.class_names


# Training 

In [65]:
data_augmentation = Sequential([
  RandomFlip("horizontal_and_vertical"),
  RandomRotation(0.2),
])


In [64]:
num_classes = len(class_names)

model = Sequential([
  data_augmentation,
  Resizing(),
  Rescaling(1./255),
  Conv2D(32, 3, activation='relu'),
  MaxPooling2D(),
  Conv2D(32, 3, activation='relu'),
  MaxPooling2D(),
  Conv2D(32, 3, activation='relu'),
  MaxPooling2D(),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(num_classes)
])